In [76]:
import pandas as pd
import numpy as np
import time

# Importing dataset

In [77]:
graph = pd.read_csv(
    "data/opsahl-ucsocial/out.opsahl-ucsocial", 
    names=["from", "to", "weight", "timestamp"],
    sep=" |\t",
    engine ='python',
    skiprows=[0, 1]
)
#print(graph.head())

In [78]:
V = np.unique(graph["from"]._append(graph["to"]))
n = V.size
volume = graph["timestamp"].size
print(f"{n = }, {volume = }")

n = 1899, volume = 59835


# Chapter 1

## Preparing static graph

In [79]:
graph_static = [set() for _ in range(n +1)]
for row_number, row in graph.iterrows():
    graph_static[row["from"]].add(row["to"])
    graph_static[row["to"]].add(row["from"])


## Task 1.1

In [80]:
E_count = 0
for i in range(len(graph_static)):
    E_count += len(graph_static[i])
E_count //= 2
#print(E_count)

density = E_count * 2 / (n * (n - 1))
#print(density)

In [81]:
V_to_visit = set(V)
connectivity_components = []
while(V_to_visit):
    V_visited = set()
    queue = []
    for u in V_to_visit:
        queue.append(u)
        break
    while queue:
        u = queue.pop()
        V_to_visit.discard(u)
        V_visited.add(u)
        u_adjacent_to_visit = graph_static[u].difference(V_visited)
        for v in u_adjacent_to_visit:
            if v not in queue:
                queue.append(v)
    connectivity_components.append(V_visited)

sizes = list(map(lambda x: len(x), connectivity_components))
max_val = max(sizes)
max_connectivity_component_index = sizes.index(max_val)
proportion = max_val / len(V)
#print(f"{max_val = }, {max_connectivity_component_index = }, {proportion = }")


In [82]:
# 1.1
print("|V| = %i, |E| = %i, p = %f, number of components = %i, max component proportion = %f" 
      % (n, E_count, density, len(connectivity_components), proportion))

|V| = 1899, |E| = 13838, p = 0.007679, number of components = 4, max component proportion = 0.996840


## Task 1.2

In [83]:
component = list(connectivity_components[max_connectivity_component_index])
# infinite distance = n + 1
distance_matrix = [[n + 1 for j in range(n + 1)] for i in range(n + 1)]
eccentricities = dict()
for start in component:
    V_visited = set()
    queue = [(start, 0)]
    queued = set([start])
    max_depth = 0
    while queue:
        u, depth = queue.pop(0)
        max_depth = max(max_depth, depth)
        queued.discard(u)
        V_visited.add(u)
        u_adjacent_to_visit = graph_static[u].difference(V_visited)
        for v in u_adjacent_to_visit:
            if v not in queued:
                distance = distance_matrix[start][v]
                if depth + 1 < distance:
                    distance_matrix[start][v] = depth + 1
                    distance_matrix[v][start] = depth + 1
                queue.append((v, depth + 1))
                queued.add(v)
    eccentricities[start] = max_depth

In [84]:
component = set(connectivity_components[max_connectivity_component_index])

diameter = max(eccentricities.values())
radius = min(eccentricities.values())

all_distances = []
for i in range(1, n + 1):
    for j in range(i + 1, n + 1):
        if i in component and j in component:
            all_distances.append(distance_matrix[i][j])
all_distances.sort()

In [93]:
percentile_90 = np.percentile(all_distances, 90)

In [86]:
import random

component = list(connectivity_components[max_connectivity_component_index])

random_500_vertices = sorted(random.sample(component, 500))
random_1000_vertices = sorted(random.sample(component, 1000))


def get_random_distances(vertices):
    distances = []
    for i in range(len(vertices) - 1):
        for j in range(i + 1, len(vertices)):
            u = vertices[i]
            v = vertices[j]
            distances.append(distance_matrix[u][v])
    return distances

random_500_eccentricities = [eccentricities[v] for v in random_500_vertices]
random_1000_eccentricities = [eccentricities[v] for v in random_1000_vertices]

random_500_distances = get_random_distances(random_500_vertices)    
random_1000_distances = get_random_distances(random_1000_vertices)  

diameter_from_random_500 = max(random_500_eccentricities)
radius_from_random_500 = min(random_500_eccentricities)
percentile_90_from_random_500 = np.percentile(random_500_distances, 90)

print(f"{diameter_from_random_500 = }, {radius_from_random_500 = }, {percentile_90_from_random_500 = }")

diameter_from_random_1000 = max(random_1000_eccentricities)
radius_from_random_1000 = min(random_1000_eccentricities)
percentile_90_from_random_1000 = np.percentile(random_1000_distances, 90)

print(f"{diameter_from_random_1000 = }, {radius_from_random_1000 = }, {percentile_90_from_random_1000 = }")

                    

diameter_from_random_500 = 8, radius_from_random_500 = 4, percentile_90_from_random_500 = 4.0
diameter_from_random_1000 = 8, radius_from_random_1000 = 4, percentile_90_from_random_1000 = 4.0


In [87]:
def snowball(limit):
    vertices = {1, 2}
    while len(vertices) < limit:
        for v in vertices:
            if len(vertices) < limit:
                vertices = vertices.union(graph_static[v])
    return sorted(list(vertices))


snowball_vertices_500 = snowball(500)
snowball_vertices_1000 = snowball(1000)


snowball_500_eccentricities = [eccentricities[v] for v in snowball_vertices_500]
snowball_1000_eccentricities = [eccentricities[v] for v in snowball_vertices_1000]

snowball_500_distances = get_random_distances(snowball_vertices_500)    
snowball_1000_distances = get_random_distances(snowball_vertices_1000)  

diameter_from_snowball_500 = max(snowball_500_eccentricities)
radius_from_snowball_500 = min(snowball_500_eccentricities)
percentile_90_from_snowball_500 = np.percentile(snowball_500_distances, 90)

print(f"{diameter_from_snowball_500 = }, {radius_from_snowball_500 = }, {percentile_90_from_snowball_500 = }")

diameter_from_snowball_1000 = max(snowball_1000_eccentricities)
radius_from_snowball_1000 = min(snowball_1000_eccentricities)
percentile_90_from_snowball_1000 = np.percentile(snowball_1000_distances, 90)

print(f"{diameter_from_snowball_1000 = }, {radius_from_snowball_1000 = }, {percentile_90_from_snowball_1000 = }")


diameter_from_snowball_500 = 6, radius_from_snowball_500 = 4, percentile_90_from_snowball_500 = 3.0
diameter_from_snowball_1000 = 6, radius_from_snowball_1000 = 4, percentile_90_from_snowball_1000 = 3.0


In [94]:
# 1.2
print("diameter = %i, raduis = %i, percentile_90 = %i" 
      % (diameter, radius, percentile_90))

diameter = 8, raduis = 4, percentile_90 = 4


## Task 1.3

In [89]:
component = list(connectivity_components[max_connectivity_component_index])

Cl = dict()
for u in component:
    u_neighbors = graph_static[u]

    if len(u_neighbors) < 2:
        Cl[u] = 0
        continue

    Lu_doubled = 0
    for neighbor in u_neighbors:
        Lu_doubled += len(graph_static[neighbor].intersection(u_neighbors))
    Cl[u] = Lu_doubled / (len(u_neighbors) * (len(u_neighbors) - 1))

Cl_average = sum(Cl.values()) / len(Cl.values())

In [90]:
#1.3
print("Cl_average = %f" % (Cl_average))

Cl_average = 0.109746


## Task 1.4

In [91]:
R1 = 0
R2 = 0
R3 = 0
Re = 0
for i in range(1, n + 1):
    ki = len(graph_static[i])
    R1 += ki
    R2 += ki**2
    R3 += ki**3
    for j in range(1, n + 1):
        if j in graph_static[i]:
            kj = len(graph_static[j])
            Re += ki * kj
degree_associativity = (Re * R1 - R2**2) / (R3 * R1 - R2**2)

In [92]:
print("Degree associativity = %i" % (degree_associativity))

Degree associativity = 0
